<a href="https://colab.research.google.com/github/SecrezS/MACDforecasting/blob/main/XLK_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Step 1: Import required libraries

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split

# Download and extract the libta-lib binaries
url = 'https://anaconda.org/conda-forge/libta-lib/0.4.0/download/linux-64/libta-lib-0.4.0-h166bdaf_1.tar.bz2'
!curl -L $url | tar xj -C /usr/lib/x86_64-linux-gnu/ lib --strip-components=1

# Download and extract the Python wrapper for TA-Lib
url = 'https://anaconda.org/conda-forge/ta-lib/0.4.19/download/linux-64/ta-lib-0.4.19-py310hde88566_4.tar.bz2'
!curl -L $url | tar xj -C /usr/local/lib/python3.10/dist-packages/ lib/python3.10/site-packages/talib --strip-components=3

# Import TA-Lib
import talib

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  4055    0  4055    0     0   9821      0 --:--:-- --:--:-- --:--:--  9818
100  517k  100  517k    0     0   386k      0  0:00:01  0:00:01 --:--:--  386k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  4039    0  4039    0     0   8515      0 --:--:-- --:--:-- --:--:--  8575
100  392k  100  392k    0     0   331k      0  0:00:01  0:00:01 --:--:--  859k


# Step 2: Download data and prepare main dataframe

In [2]:
import yfinance as yf

#Load dataset

# Download data for multiple tickers (SPY, DIA, QQQ, IWM, XLK)
tickers = ["SPY", "DIA", "QQQ", "IWM", "XLK"]
dfs = {}

for ticker in tickers:
    # Download historical data without multi-index column names
    df = yf.download(ticker, start="2012-01-01", end="2024-01-01")
    df.reset_index(inplace=True)

    # Rename columns to remove multi-index formatting and ticker name
    df.columns = [col[0] if isinstance(col, tuple) else col for col in df.columns]

    # Store the cleaned DataFrame in the dictionary
    dfs[ticker] = df

# Choose the main dataset for demonstration, XLK
data = dfs["XLK"].copy()

data.head()

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


,Date,Adj Close,Close,High,Low,Open,Volume
0,2012-01-03 00:00:00+00:00,21.622305,25.809999,26.020000,25.790001,25.870001,26839900
1,2012-01-04 00:00:00+00:00,21.680956,25.879999,25.930000,25.660000,25.799999,4744300
2,2012-01-05 00:00:00+00:00,21.747972,25.959999,26.010000,25.730000,25.850000,6756900
3,2012-01-06 00:00:00+00:00,21.756353,25.969999,26.030001,25.850000,25.980000,4989200
4,2012-01-09 00:00:00+00:00,21.722841,25.930000,26.080000,25.860001,26.059999,4086600


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3018 entries, 0 to 3017
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype              
---  ------     --------------  -----              
 0   Date       3018 non-null   datetime64[ns, UTC]
 1   Adj Close  3018 non-null   float64            
 2   Close      3018 non-null   float64            
 3   High       3018 non-null   float64            
 4   Low        3018 non-null   float64            
 5   Open       3018 non-null   float64            
 6   Volume     3018 non-null   int64              
dtypes: datetime64[ns, UTC](1), float64(5), int64(1)
memory usage: 165.2 KB


#Step 3: Adding features (technical indicators)

In [4]:
# Calculate RSI
data['RSI'] = talib.RSI(data['Adj Close'], timeperiod=14)

# Calculate Moving Average Convergence Divergence (MACD)
data['MACD'], data['MACD_signal'], data['MACD_diff'] = talib.MACD(data['Adj Close'], fastperiod=12, slowperiod=26, signalperiod=9)

# Calculate Average True Range (ATR)
data['ATR'] = talib.ATR(data['High'], data['Low'], data['Adj Close'], timeperiod=14)

# Calculate Bollinger Bands (BB)
data['bb_bbm'], data['bb_bbh'], data['bb_bbl'] = talib.BBANDS(data['Adj Close'], timeperiod=20, nbdevup=2, nbdevdn=2, matype=0)

# Calculate Keltner Channel (KC)
data['EMA_KC'] = talib.EMA(data['Adj Close'], timeperiod=20)
data['KC_upper'] = data['EMA_KC'] + (data['ATR'] * 1.5)
data['KC_lower'] = data['EMA_KC'] - (data['ATR'] * 1.5)
data['KC'] = (data['Adj Close'] - data['KC_upper']) / data['Adj Close']
data.drop(columns=['EMA_KC', 'KC_upper', 'KC_lower'], inplace=True)

# Calculate Donchian Channel (DC)
data['DC_upper'] = data['High'].rolling(window=20).max()
data['DC_lower'] = data['Low'].rolling(window=20).min()
data['DC_middle'] = (data['DC_upper'] + data['DC_lower']) / 2

# Calculate On-Balance Volume (OBV)
data['OBV'] = talib.OBV(data['Adj Close'], data['Volume'])

# Calculate Chaikin Money Flow (CMF)
cmf_period = 20
data['Money_Flow_Multiplier'] = ((data['Adj Close'] - data['Low']) - (data['High'] - data['Adj Close'])) / (data['High'] - data['Low'])
data['Money_Flow_Volume'] = data['Money_Flow_Multiplier'] * data['Volume']
data['CMF'] = data['Money_Flow_Volume'].rolling(window=cmf_period).sum() / data['Volume'].rolling(window=cmf_period).sum()
data.drop(columns=['Money_Flow_Multiplier', 'Money_Flow_Volume'], inplace=True)

# Calculate the Stochastic Oscillator
data['stoch_k'], data['stoch_d'] = talib.STOCH(data['High'], data['Low'], data['Adj Close'],
                                               fastk_period=14,  # The %K period
                                               slowk_period=3,   # The %K slowing period
                                               slowk_matype=0,   # Moving average type for %K (0 = SMA)
                                               slowd_period=3,   # The %D period
                                               slowd_matype=0)   # Moving average type for %D (0 = SMA)

# Calculate the Average Directional Index (ADX)
data['ADX'] = talib.ADX(data['High'], data['Low'], data['Adj Close'], timeperiod=14)

# Number of lag days to create
lags = 20

# List of features to create lag features (exlcude "Adj Close")
features_to_lag = ['Volume', 'RSI', 'MACD', 'MACD_signal',
                   'MACD_diff', 'ATR', 'bb_bbm', 'bb_bbh',
                   'bb_bbl', 'KC','DC_upper', 'DC_lower',
                   'DC_middle','OBV','CMF','stoch_k','stoch_d','ADX']

# Function to create lagged features
def create_lagged_features(df, lags, columns):
    for col in columns:
        for lag in range(1, lags + 1):
            df[f'{col}_lag_{lag}'] = df[col].shift(lag)
    return df

# Apply lagged features
data = create_lagged_features(data, lags, features_to_lag)
data.dropna(inplace=True)

# Display the data to see the added lagged columns
data.head()

<ipython-input-4-b247fe6fb666>:59: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f'{col}_lag_{lag}'] = df[col].shift(lag)
<ipython-input-4-b247fe6fb666>:59: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f'{col}_lag_{lag}'] = df[col].shift(lag)
<ipython-input-4-b247fe6fb666>:59: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, us

,Date,Adj Close,Close,High,Low,Open,Volume,RSI,MACD,MACD_signal,...,ADX_lag_11,ADX_lag_12,ADX_lag_13,ADX_lag_14,ADX_lag_15,ADX_lag_16,ADX_lag_17,ADX_lag_18,ADX_lag_19,ADX_lag_20
53,2012-03-20 00:00:00+00:00,25.272018,30.070000,30.110001,29.840000,29.920000,5713100,79.086860,0.498565,0.460674,...,59.961037,62.287484,62.859545,63.520825,64.232973,65.276805,66.700161,66.718622,67.027808,66.278047
54,2012-03-21 00:00:00+00:00,25.263615,30.059999,30.250000,30.049999,30.120001,8825000,78.695571,0.497402,0.468020,...,56.218064,59.961037,62.287484,62.859545,63.520825,64.232973,65.276805,66.700161,66.718622,67.027808
55,2012-03-22 00:00:00+00:00,25.204782,29.990000,30.070000,29.879999,29.930000,7500900,75.865719,0.486130,0.471642,...,53.374805,56.218064,59.961037,62.287484,62.859545,63.520825,64.232973,65.276805,66.700161,66.718622
56,2012-03-23 00:00:00+00:00,25.204782,29.990000,30.040001,29.830000,30.040001,9511600,75.865719,0.471758,0.471665,...,51.594630,53.374805,56.218064,59.961037,62.287484,62.859545,63.520825,64.232973,65.276805,66.700161
57,2012-03-26 00:00:00+00:00,25.574575,30.430000,30.440001,30.120001,30.190001,12187200,80.878224,0.484621,0.474256,...,50.185142,51.594630,53.374805,56.218064,59.961037,62.287484,62.859545,63.520825,64.232973,65.276805


#Step 4: Data Cleaning

In [5]:
#Drop Non-Lagged Technical Indicators

# Define the target column
target_column = 'Adj Close'

# List of non-lagged technical indicators to drop
non_lagged_technical_indicators = [
    'RSI', 'MACD', 'MACD_signal', 'MACD_diff', 'ATR', 'bb_bbm', 'bb_bbh', 'bb_bbl',
    'KC', 'DC_upper', 'DC_lower', 'DC_middle', 'OBV', 'CMF', 'ADX', 'stoch_k', 'stoch_d'
]

# Drop non-lagged columns and other unwanted columns
columns_to_drop = ['Open', 'High', 'Low', 'Close', 'Volume'] + non_lagged_technical_indicators
data = data.drop(columns=columns_to_drop, errors='ignore')

#Step 5: Feature Importance

In [6]:
# Train RandomForestRegressor to determine feature importance
X = data.drop(columns=['Adj Close','Date'])
y = data['Adj Close']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
forest = RandomForestRegressor(n_estimators=100, random_state=42)
forest.fit(X_train, y_train)
importances = forest.feature_importances_
indices = np.argsort(importances)[::-1]
important_features = [X.columns[i] for i in indices if importances[i] > 0.01]

#Step 6: Split data

In [7]:
# Convert 'Date' to datetime and set as index (if not already done)
data['Date'] = pd.to_datetime(data['Date'])
data.set_index('Date', inplace=True)

# Now split the data into train, validation, and test sets
train_data = data[:'2021-12-31']
val_data = data['2022-01-01':'2022-12-31']
test_data = data['2023-01-01':]

# Verify splits by printing the outputs (optional step to confirm the splits are correct)
print("Training Data Sample:", train_data.head())
print("Validation Data Sample:", val_data.head())
print("Testing Data Sample:", test_data.head())

# Prepare features and targets using only important features
X_train = train_data[important_features]
y_train = train_data['Adj Close']
X_val = val_data[important_features]
y_val = val_data['Adj Close']
X_test = test_data[important_features]
y_test = test_data['Adj Close']

Training Data Sample:                            Adj Close  Volume_lag_1  Volume_lag_2  \
Date                                                               
2012-03-20 00:00:00+00:00  25.272018     8930800.0     9478800.0   
2012-03-21 00:00:00+00:00  25.263615     5713100.0     8930800.0   
2012-03-22 00:00:00+00:00  25.204782     8825000.0     5713100.0   
2012-03-23 00:00:00+00:00  25.204782     7500900.0     8825000.0   
2012-03-26 00:00:00+00:00  25.574575     9511600.0     7500900.0   

                           Volume_lag_3  Volume_lag_4  Volume_lag_5  \
Date                                                                  
2012-03-20 00:00:00+00:00    12450500.0    12741800.0    19012100.0   
2012-03-21 00:00:00+00:00     9478800.0    12450500.0    12741800.0   
2012-03-22 00:00:00+00:00     8930800.0     9478800.0    12450500.0   
2012-03-23 00:00:00+00:00     5713100.0     8930800.0     9478800.0   
2012-03-26 00:00:00+00:00     8825000.0     5713100.0     8930800.0   

   

#Step 7: Scale the data

In [8]:
# Scale the data
scaler_X = MinMaxScaler()
scaler_y = MinMaxScaler()

X_train_scaled = scaler_X.fit_transform(X_train)
X_val_scaled = scaler_X.transform(X_val)
X_test_scaled = scaler_X.transform(X_test)

y_train_scaled = scaler_y.fit_transform(y_train.values.reshape(-1, 1))
y_val_scaled = scaler_y.transform(y_val.values.reshape(-1, 1))
y_test_scaled = scaler_y.transform(y_test.values.reshape(-1, 1))

#Step 8: Build and Train the LSTM Model

In [9]:
# Build and train LSTM using important features
model = Sequential([
    LSTM(50, return_sequences=True, input_shape=(1, len(important_features))),
    Dropout(0.2),
    LSTM(50, return_sequences=False),
    Dropout(0.2),
    Dense(25),
    Dense(1)
])
model.compile(optimizer='adam', loss='mean_squared_error')
model.fit(X_train_scaled.reshape((-1, 1, len(important_features))), y_train_scaled, epochs=50, batch_size=32, validation_data=(X_val_scaled.reshape((-1, 1, len(important_features))), y_val_scaled))

Epoch 1/50


/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


77/77 ━━━━━━━━━━━━━━━━━━━━ 5s 12ms/step - loss: 0.0624 - val_loss: 0.0060
Epoch 2/50
77/77 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 0.0026 - val_loss: 0.0024
Epoch 3/50
77/77 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0020 - val_loss: 0.0021
Epoch 4/50
77/77 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0014 - val_loss: 0.0043
Epoch 5/50
77/77 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0013 - val_loss: 0.0021
Epoch 6/50
77/77 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 0.0012 - val_loss: 0.0024
Epoch 7/50
77/77 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0010 - val_loss: 0.0021
Epoch 8/50
77/77 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0011 - val_loss: 0.0019
Epoch 9/50
77/77 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0012 - val_loss: 0.0021
Epoch 10/50
77/77 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 0.0011 - val_loss: 0.0023
Epoch 11/50
77/77 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0014 - val_loss: 0.0033
Epoch 12/50
77/77 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 9.6724e-04 - val_loss: 0.

#Step 9: Make Predictions and Evaluate

In [10]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, mean_absolute_percentage_error
import numpy as np
import matplotlib.pyplot as plt
# Make predictions and evaluate the model
predictions = model.predict(X_test_scaled.reshape((-1, 1, len(important_features))))
predictions = scaler_y.inverse_transform(predictions)
actual = scaler_y.inverse_transform(y_test_scaled.reshape(-1, 1))

# Calculate performance metrics
rmse = np.sqrt(mean_squared_error(actual, predictions))
mse = mean_squared_error(actual, predictions)
mae = mean_absolute_error(actual, predictions)
mape = mean_absolute_percentage_error(actual, predictions)
r2 = r2_score(actual, predictions)

8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 46ms/step


MAPE: 0.0569, MAE: 9.4602, MSE: 127.0238, RMSE: 11.2705, R²: 0.5910


In [11]:
# Print performance metrics and plot results
print(f"MAPE: {mape:.4f}, MAE: {mae:.4f}, MSE: {mse:.4f}, RMSE: {rmse:.4f}, R²: {r2:.4f}")


plt.figure(figsize=(10, 5))
plt.plot(test_data.index, actual, label='Actual')
plt.plot(test_data.index, predictions, label='Predicted')
plt.title('Actual vs Predicted')
plt.legend()
plt.show()

In [12]:
X_test

,bb_bbl_lag_1,DC_lower_lag_1,DC_middle_lag_1,bb_bbm_lag_1,bb_bbl_lag_2,bb_bbh_lag_1,bb_bbm_lag_20
Date,,,,,,,
2023-01-03 00:00:00+00:00,118.675812,121.290001,130.310001,134.645360,118.858854,126.660586,137.556440
2023-01-04 00:00:00+00:00,118.384228,121.290001,130.310001,133.781787,118.675812,126.083008,137.056432
2023-01-05 00:00:00+00:00,118.059880,121.290001,130.310001,133.203816,118.384228,125.631848,136.500133
2023-01-06 00:00:00+00:00,117.274137,120.970001,130.150002,133.130816,118.059880,125.202476,135.989129
2023-01-09 00:00:00+00:00,117.068368,120.809998,130.070000,132.892873,117.274137,124.980621,134.078020
...,...,...,...,...,...,...,...
2023-12-22 00:00:00+00:00,179.466112,181.320007,187.180000,192.828224,179.354656,186.147168,190.020914
2023-12-26 00:00:00+00:00,179.699473,181.320007,187.180000,193.395288,179.466112,186.547381,189.940510
2023-12-27 00:00:00+00:00,179.967779,181.320007,187.285004,194.007987,179.699473,186.987883,189.725151
